In [ ]:
!pip install -q medspacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.4/111.4 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.4/67.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.0/580.0 kB 48.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 31.2 MB/s eta 0:00:00


Kita akan membutuhkan TargetMatcher dan TargetRule untuk NER berbasis aturan. visualize_ent memungkinkan kita untuk secara visual menandai frasa atau bagian mana dalam teks yang berhubungan dengan entitas tertentu. Visualize_dep untuk memvisualisasikan ketergantungan di antara entitas yang berbeda dalam kalimat.

In [ ]:
import sys
import spacy
import medspacy

from medspacy.ner import TargetMatcher, TargetRule
from medspacy.visualization import visualize_ent, visualize_dep

In [ ]:
with open("discharge_summary.txt") as f:
    text = f.read()

In [ ]:
# load model
nlp = medspacy.load(medspacy_enable=["medspacy_pyrush"])

In [ ]:
nlp.pipe_names

['medspacy_pyrush']

## Ekstraksi target
Pada langkah ini, kita akan menulis aturan untuk mengekstrak konsep utama yang kita minati.

Dalam contoh ini, kita akan menggunakan dua utilitas yang disediakan di medspacy.ner untuk pencocokan berbasis aturan: TargetMatcher dan TargetRule. Namun, Anda dapat menggunakan komponen spaCy apa pun untuk menambahkan rentang ke doc.ents, termasuk model NER yang telah dilatih sebelumnya atau komponen pencocokan berbasis aturan spaCy lainnya.

Jika Anda ingin menggunakan model NER statistik alih-alih komponen berbasis aturan, lihat [./6-Menggunakan-Model-Pretrained.ipynb](https://github.com/medspacy/medspacy/blob/master/notebooks/03-Information-Extraction.ipynb)

### Konsep target
Dalam teks kita, kita akan mengekstrak konsep-konsep berikut:
- Diagnosa
- Pengobatan

Selain itu, kami akan menunjukkan beberapa contoh cara menambahkan atribut spaCy khusus ke aturan target untuk menambahkan kode diagnosis ICD-10 sebagai atribut entitas.

Pertama, kita akan mengimpor TargetMatcher dan TargetRule. Kita akan menginstansiasi TargetRule dan menambahkannya ke pipeline kita.

In [ ]:
target_matcher = nlp.add_pipe("medspacy_target_matcher")

Sekarang kita akan mendefinisikan beberapa aturan untuk mengekstrak konsep dari teks. Kelas TargetRule mendefinisikan aturan untuk mengekstrak entitas dari teks. TargetRule mengambil argumen berikut:

- literal: Frasa yang tepat untuk dicocokkan dalam teks
- kategori: Kelas semantik dari entitas, akan sesuai dengan ent.label_ di dalam dokumen Doc.
- pola: Pola opsional untuk dicocokkan, bukan literal. Seperti yang akan kita lihat di bawah ini, pola ini dapat berupa daftar kamus yang mendefinisikan atribut token atau string ekspresi reguler.
- on_match: Fungsi pemanggilan balik opsional. Lihat https://spacy.io/usage/rule-based-matching#on_match
- metadata: Kamus metadata yang bersifat opsional
- atribut: Kamus opsional dari atribut khusus untuk ditetapkan dalam entitas yang dihasilkan.

Kita akan melihat contoh di bawah ini.
Mari kita tentukan beberapa aturan sederhana:

In [ ]:
target_rules = [
    TargetRule(literal="abdominal pain", category="PROBLEM"),
    TargetRule("stroke", "PROBLEM"),
    TargetRule("hemicolectomy", "TREATMENT"),
    TargetRule("Hydrochlorothiazide", "TREATMENT"),
    TargetRule("colon cancer", "PROBLEM"),
    TargetRule("metastasis", "PROBLEM"),

]

In [ ]:
text

"Admission Date:  [**2573-5-30**]              Discharge Date:   [**2573-7-1**]\n\nDate of Birth:  [**2498-8-19**]             Sex:   F\n\nService: SURGERY\n\nAllergies:\nHydrochlorothiazide\n\nAttending:[**First Name3 (LF) 1893**]\nChief Complaint:\nAbdominal pain\n\nMajor Surgical or Invasive Procedure:\nPICC line [**6-25**]\nERCP w/ sphincterotomy [**5-31**]\n\n\nHistory of Present Illness:\n74y female with type 2 dm and a recent stroke affecting her\nspeech, who presents with 2 days of abdominal pain. Imaging shows no evidence of metastasis. She is not receiving any chemo.\n\nPast Medical History:\n1. Colon cancer dx'd in [**2554**], tx'd with hemicolectomy, XRT,\nchemo. Last colonoscopy showed: Last CEA was in the 8 range\n(down from 9)\n2. Type II Diabetes Mellitus\n3. Hypertension\n\nSocial History:\nMarried, former tobacco use. No alcohol or drug use.\n\nFamily History:\nMother with stroke at age 82. no early deaths.\n2 daughters- healthy\n\n\nBrief Hospital Course:\nMs. [**Kno

In [ ]:
target_matcher.add(target_rules)
doc = nlp(text)

In [ ]:
print(doc.ents)

(Hydrochlorothiazide, Abdominal pain, stroke, abdominal pain, metastasis, Colon cancer, hemicolectomy, stroke, abdominal pain, abdominal pain)


In [ ]:
for ent in doc.ents:
    print(ent, ent.label_, ent._.target_rule, sep="  |  ")

Hydrochlorothiazide  |  TREATMENT  |  TargetRule(literal="Hydrochlorothiazide", category="TREATMENT", pattern=None, attributes=None, on_match=None)
Abdominal pain  |  PROBLEM  |  TargetRule(literal="abdominal pain", category="PROBLEM", pattern=None, attributes=None, on_match=None)
stroke  |  PROBLEM  |  TargetRule(literal="stroke", category="PROBLEM", pattern=None, attributes=None, on_match=None)
abdominal pain  |  PROBLEM  |  TargetRule(literal="abdominal pain", category="PROBLEM", pattern=None, attributes=None, on_match=None)
metastasis  |  PROBLEM  |  TargetRule(literal="metastasis", category="PROBLEM", pattern=None, attributes=None, on_match=None)
Colon cancer  |  PROBLEM  |  TargetRule(literal="colon cancer", category="PROBLEM", pattern=None, attributes=None, on_match=None)
hemicolectomy  |  TREATMENT  |  TargetRule(literal="hemicolectomy", category="TREATMENT", pattern=None, attributes=None, on_match=None)
stroke  |  PROBLEM  |  TargetRule(literal="stroke", category="PROBLEM", pa

In [ ]:
## Source: https://github.com/medspacy/medspacy

# Load medspacy model
nlp = medspacy.load()
print(nlp.pipe_names)

text = """
Past Medical History:
1. Atrial fibrillation
2. Type II Diabetes Mellitus

Assessment and Plan:
There is no evidence of pneumonia. Continue warfarin for Afib. Follow up for management of type 2 DM.
"""

# Add rules for target concept extraction
target_matcher = nlp.get_pipe("medspacy_target_matcher")
target_rules = [
    TargetRule("atrial fibrillation", "PROBLEM"),
    TargetRule("atrial fibrillation", "PROBLEM", pattern=[{"LOWER": "afib"}]),
    TargetRule("pneumonia", "PROBLEM"),
    TargetRule("Type II Diabetes Mellitus", "PROBLEM",
              pattern=[
                  {"LOWER": "type"},
                  {"LOWER": {"IN": ["2", "ii", "two"]}},
                  {"LOWER": {"IN": ["dm", "diabetes"]}},
                  {"LOWER": "mellitus", "OP": "?"}
              ]),
    TargetRule("warfarin", "MEDICATION")
]
target_matcher.add(target_rules)

doc = nlp(text)
visualize_ent(doc)

['medspacy_pyrush', 'medspacy_target_matcher', 'medspacy_context']
